Import libraries

In [19]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

Load the dataset

In [20]:
# Folder path 
folder_path = 'D:\FPTUni\SP24\ADY201m\Lab05\ADY201m_Lab05_SE183256'
# Import the dataset
train_data = pd.read_csv(os.path.join(folder_path, 'train.csv'))
test_data = pd.read_csv(os.path.join(folder_path, 'test.csv'))
output_path = os.path.join(folder_path, 'submission.csv')

In [21]:
# Save ID of test data
test_id = test_data['Id']

In [22]:
# Drop unnecessary columns
train_data = train_data.drop('Id', axis=1)

House Price Distribution

In [23]:
# Setup threshold for missing values

# === IMPORTANT ===


# Here we will put the variable to choose the threshold
threshold = 0.5


# === IMPORTANT ===

Numerical data distribution

In [24]:
# Because MSSubClass is int64 but it is a categorical variable, so we need to convert it to string
train_data['MSSubClass'] = train_data['MSSubClass'].astype(str)

In [25]:
# Get the categorical columns
categorical_cols = train_data.select_dtypes(include=['object']).columns.tolist()
# Delete all categorical columns
train_data = train_data.drop(categorical_cols, axis=1)

In [26]:
# Compute the correlation matrix with SalePrice
corr_matrix = train_data.corr()
corr_matrix['SalePrice'].sort_values(ascending=False)
# ABS of correlation coefficient
# Get the features that have the correlation coefficient with SalePrice greater than the threshold
selected_features = corr_matrix['SalePrice'][corr_matrix['SalePrice'].abs() > threshold].index.tolist()
# Display the selected features

In [27]:
# Create the scaler
scaler = StandardScaler()
scaler2 = StandardScaler()
# Fit the scaler to the train data
scaler.fit(train_data[selected_features])
# Transform the train data
train_data_scaled = scaler.transform(train_data[selected_features])

In [28]:
# Convert test data to same format with train data to predict, ( test data doest not have SalePrice column)
# remove saleprice column from selected_features and make new selected_features2
selected_features2 = selected_features.copy()
selected_features2.remove('SalePrice')
# if Id have in test data, we need to remove it, if not, we don't need to remove it
if 'Id' in test_data.columns:
    test_data = test_data.drop('Id', axis=1)
scaler2.fit(test_data[selected_features2])
test_data_scaled = scaler2.transform(test_data[selected_features2])

In [29]:
# Prepare the data
y_train = train_data_scaled[:, -1]
x_train = train_data_scaled[:, :-1]

# Split the train data into train and test data
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [30]:
# Create the model
model01 = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(len(selected_features) - 1,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'), 
    layers.Dense(1, activation='linear')
])

# Compile the model
model01.compile(optimizer='adam', loss='mse', metrics=['mae'])
# Train the model
model01.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/100


37/37 [==============================] - 1s 5ms/step - loss: 0.3836 - mae: 0.4357 - val_loss: 0.2018 - val_mae: 0.2775
Epoch 2/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1972 - mae: 0.2715 - val_loss: 0.1811 - val_mae: 0.2561
Epoch 3/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1758 - mae: 0.2620 - val_loss: 0.1682 - val_mae: 0.2621
Epoch 4/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1703 - mae: 0.2549 - val_loss: 0.1904 - val_mae: 0.2525
Epoch 5/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1684 - mae: 0.2497 - val_loss: 0.2599 - val_mae: 0.2851
Epoch 6/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1583 - mae: 0.2539 - val_loss: 0.1483 - val_mae: 0.2465
Epoch 7/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1465 - mae: 0.2383 - val_loss: 0.1631 - val_mae: 0.2426
Epoch 8/100
37/37 [==============================] - 0s 2ms/step - loss: 0.1338 - mae: 0.2303 -

In [31]:
# Save model to file
# model01.save('model01.house_price_prediction')

In [32]:
# Predict using the trained model
result_scaled = model01.predict(test_data_scaled)
# Combine scaled test data with scaled result
combined_data_scaled = np.concatenate((test_data_scaled, result_scaled), axis=1)
# Unscale the combined data to get the final result
result = scaler.inverse_transform(combined_data_scaled)[:, -1]

46/46 [==============================] - 0s 847us/step


In [33]:
# Create the submission file
submission = pd.DataFrame({'Id': test_id, 'SalePrice': result})
# Save the submission file
submission.to_csv(output_path, index=False)

In [34]:
# Predict using the trained model
result_scaled = model01.predict(x_train)
# Unscale the combined data to get the final result
result = scaler.inverse_transform(np.concatenate((x_train, result_scaled), axis=1))[:, -1]
y_train_unscaled = scaler.inverse_transform(np.concatenate((x_train, y_train.reshape(-1, 1)), axis=1))[:, -1]
# Display random 100 results
for i in range(10):
    print(f'Predicted: {result[i]}, Actual: {y_train_unscaled[i]}')

37/37 [==============================] - 0s 1ms/step
Predicted: 136833.8934590456, Actual: 145000.0
Predicted: 179954.39197955275, Actual: 178000.0
Predicted: 85088.87235357052, Actual: 85000.0
Predicted: 174461.05843442984, Actual: 175000.0
Predicted: 126077.05386564496, Actual: 127000.0
Predicted: 131981.56183785124, Actual: 149900.0
Predicted: 173659.86752830193, Actual: 174000.0
Predicted: 126698.56507612474, Actual: 125500.0
Predicted: 174695.64282329395, Actual: 175500.0
Predicted: 260852.71602927742, Actual: 225000.0
